In [1]:
import scipy
import scipy.io as sio
import numpy as np
from PIL import Image
import math
import os
from keras.models import Sequential
from keras.layers import *
from keras.layers.advanced_activations import PReLU
#utilities help us transform our data
from keras.utils import * 
from sklearn.cross_validation import train_test_split
#rows, cols = 64,64
import pandas as pd
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import tensorflow as tf

import cv2
from keras.models import load_model
from statistics import mode
#from utils import preprocess_input
from random import shuffle

Using TensorFlow backend.


In [32]:
class Data_Generator:
    def __init__(self, metadata_path, batch_size, val_split):

        self.data = scipy.io.loadmat(metadata_path)
        self.img_paths = self.data['wiki']['full_path'][0][0][0]
        self.genders = self.data['wiki']['gender'][0][0][0]
        self.batch_size = batch_size
        self.val_split = val_split  
        self.face_score = self.data['wiki']['face_score'][0][0][0]
        self.second_face_score = self.data['wiki']['second_face_score'][0][0][0]
        self.load_keys()

    def load_keys(self):
        
        """ load wiki dataset 
        # Take 5000 images of males and females
                Faces are selected with some threshold
                As all images in database are not accurate
                for gender "0" represents female and "1" represents male and "nan" represents blank image                
        """ 
        indices = []
        count_male, count_female = 0, 0
        n_m, n_f = 5000,5000
        for i in range(len(self.img_paths)):
            if count_female == n_f and count_male == n_m:
                break            
            if (self.face_score[i] > 3.00000000000 and (math.isnan(self.second_face_score[i]) == True)):
                if (math.isinf(self.face_score[i]) == False):
                    if (math.isnan(self.genders[i]) == False):
                        gender = self.genders[i]
                        if gender == 0 and count_female < n_f:
                            count_female += 1
                            indices.append(i)
                        elif gender == 1 and count_male < n_m:
                            count_male += 1
                            indices.append(i)  
                               
        self.number_of_imgs = len(indices)

        self.val_size = int(self.number_of_imgs * self.val_split)
        self.train_size = self.number_of_imgs - self.val_size
        shuffle(indices)
        self.train_keys = indices[:self.train_size]
        self.val_keys = indices[self.train_size:]


    def load_data(self, is_train):
        
        """ load wiki dataset 
        # Returns: faces and genders
                face: shape (64, 64, 1)
                gender_labels: 0 for female and 1 for male
        """ 
        
        while 1:
            faces = []
            gender_labels = []   

            shuffle(self.train_keys)
            keys = self.train_keys
            if not is_train:
                keys = self.val_keys

            for key in keys:
                img_path = self.img_paths[key][0]
                img = cv2.imread('./wiki_crop/' + img_path, 0)            

                faces.append(cv2.resize(img, (64, 64)))
                gender_labels.append(self.genders[key])

                if len(faces) == self.batch_size:
                    faces = np.expand_dims(faces,-1)
                    gender_labels = pd.get_dummies(gender_labels).as_matrix()
                    yield (faces, gender_labels)
                    faces = []
                    gender_labels = []

            if len(faces) == 0:
                continue
            faces = np.expand_dims(faces,-1)
            gender_labels = pd.get_dummies(gender_labels).as_matrix()
            yield (faces, gender_labels)


In [33]:

data_path = './wiki_crop/wiki.mat'
model_save_path = './wiki_crop/test_CNN.hdf5'
num_classes = 2
image_size = (64, 64, 1)
batch_size = 150
num_epochs = 20
val_split = 0.1
data = Data_Generator(data_path, batch_size, val_split)
gen_train = data.load_data(True)
gen_val = data.load_data(False)

In [35]:
model = CNN(image_size, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy',
                                        metrics=['accuracy'])
csv_logger = CSVLogger('training.log')
early_stop = EarlyStopping('val_acc', patience=200, verbose=1)
model_checkpoint = ModelCheckpoint(model_save_path,
                                    'val_acc', verbose=0,
                                    save_best_only=True)

model_callbacks = [early_stop, model_checkpoint, csv_logger]

K.get_session().run(tf.global_variables_initializer())
model.fit_generator(gen_train, nb_epoch=num_epochs, verbose=1, 
                                    validation_data=gen_val,
                                    samples_per_epoch=data.train_size,
                                    nb_val_samples=data.val_size,
                                    callbacks=model_callbacks)

Epoch 1/20
9000/9000 [==============================] - 320s - loss: 0.7500 - acc: 0.6279 - val_loss: 0.6029 - val_acc: 0.6840
Epoch 2/20
9000/9000 [==============================] - 301s - loss: 0.5690 - acc: 0.7048 - val_loss: 0.5724 - val_acc: 0.7060
Epoch 3/20
9000/9000 [==============================] - 291s - loss: 0.5302 - acc: 0.7362 - val_loss: 0.5102 - val_acc: 0.7700
Epoch 4/20
9000/9000 [==============================] - 290s - loss: 0.5049 - acc: 0.7480 - val_loss: 0.4620 - val_acc: 0.8040
Epoch 5/20
9000/9000 [==============================] - 289s - loss: 0.4788 - acc: 0.7714 - val_loss: 0.4981 - val_acc: 0.7850
Epoch 6/20
9000/9000 [==============================] - 289s - loss: 0.4509 - acc: 0.7917 - val_loss: 0.4591 - val_acc: 0.7710
Epoch 7/20
9000/9000 [==============================] - 290s - loss: 0.4217 - acc: 0.8031 - val_loss: 0.4072 - val_acc: 0.8140
Epoch 8/20
9000/9000 [==============================] - 292s - loss: 0.3899 - acc: 0.8211 - val_loss: 0.4036 - 

In [34]:
def CNN(input_shape,num_classes):
    model = Sequential()

    model.add(Convolution2D(16, 7, 7, border_mode='same',
                            input_shape=input_shape))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(5, 5),strides=(2, 2), border_mode='same'))
    model.add(Dropout(.5))

    model.add(Convolution2D(32, 5, 5, border_mode='same'))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(3, 3),strides=(2, 2), border_mode='same'))
    model.add(Dropout(.5))

    model.add(Convolution2D(32, 3, 3, border_mode='same'))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(3, 3),strides=(2, 2), border_mode='same'))
    model.add(Dropout(.5))

    model.add(Flatten())
    model.add(Dense(1028))
    model.add(PReLU())
    model.add(Dropout(0.5))
    model.add(Dense(1028))
    model.add(PReLU())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    return model

In [ ]:
classification_model_path = './wiki_crop/test_CNN.hdf5'
detection_model_path = './wiki_crop/haarcascade_frontalface_default.xml'
frame_window = 10
gender_labels = {0:'Female',1:'Male'}

face_detection = cv2.CascadeClassifier(detection_model_path)
gender_classifier = load_model(classification_model_path)
gender_window = []
video_capture = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.startWindowThread()
cv2.namedWindow('window_frame')
while True:
    _, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detection.detectMultiScale(gray,1.3,5)
    for (x,y,w,h) in faces:
        cv2.rectangle(gray,(x - (int)(0.2*w), y - (int)(0.3*h)),(x + (int)(1.2*w), y + (int)(1.2*h)),(255,0,0),2)
        face = gray[y - (int)(0.3*h) : y + (int)(1.2*h), x - (int)(0.2*w) : x + (int)(1.2*w)]
        try:
            face = cv2.resize(face, (64, 64))
        except:
            continue
        face = np.expand_dims(face,0)
        face = np.expand_dims(face,-1)
        gender_arg = np.argmax(gender_classifier.predict(face))
        gender = gender_labels[gender_arg]
        gender_window.append(gender)

        if len(gender_window) >= frame_window:
            gender_window.pop(0)
        try:
            gender_mode = mode(gender_window)
        except:
            continue
        cv2.putText(gray,gender_mode,(x,y-30), font, .7,(255,0,0),1,cv2.LINE_AA)
    try:
        cv2.imshow('window_frame', gray)
    except:
        continue

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()